<a href="https://colab.research.google.com/github/parastu/pythonpractice/blob/master/Peltarion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
from PIL import Image
np.random.seed(123)
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
import itertools

import keras
from keras.utils.np_utils import to_categorical # used for converting labels to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras import backend as K
import itertools
from keras.layers.normalization import BatchNormalization
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding

from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [3]:
!wget https://storage.googleapis.com/peltarion-ml-assignment/HAM10000/HAM10000_images_part_1.zip
!wget https://storage.googleapis.com/peltarion-ml-assignment/HAM10000/HAM10000_images_part_2.zip
!wget https://storage.googleapis.com/peltarion-ml-assignment/HAM10000/HAM10000_metadata.csv
!unzip -qq HAM10000_images_part_1.zip -d data
!unzip -qq HAM10000_images_part_2.zip -d data
!mv HAM10000_metadata.csv data/

--2019-09-29 08:37:29--  https://storage.googleapis.com/peltarion-ml-assignment/HAM10000/HAM10000_images_part_1.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.76.128, 2a00:1450:400c:c0b::80
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.76.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1366522108 (1.3G) [application/zip]
Saving to: ‘HAM10000_images_part_1.zip’

HAM10000_images_par 100%[===================>]   1.27G  77.0MB/s    in 20s     

2019-09-29 08:37:49 (66.5 MB/s) - ‘HAM10000_images_part_1.zip’ saved [1366522108/1366522108]

--2019-09-29 08:37:50--  https://storage.googleapis.com/peltarion-ml-assignment/HAM10000/HAM10000_images_part_2.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.76.128, 2a00:1450:400c:c08::80
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.76.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1403566547 (1.

In [0]:
#1. Function to plot model's validation loss and validation accuracy
def plot_model_history(model_history):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['acc'])+1),model_history.history['acc'])
    axs[0].plot(range(1,len(model_history.history['val_acc'])+1),model_history.history['val_acc'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(model_history.history['acc'])+1),len(model_history.history['acc'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()

In [0]:
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}
base_skin_dir = os.path.join('/content','data')

image_extension_pattern = '*.jpg'
image_paths = sorted((y for x in os.walk(base_skin_dir) for y in glob (os.path.join(x[0], image_extension_pattern))))

combined_img_dict = {os.path.splitext(os.path.basename(x))[0]: x
                     for x in image_paths}

In [0]:
skin_df = pd.read_csv('HAM10000.csv')

# Creating New Columns for better readability

skin_df['path'] = skin_df['image_id'].map(combined_img_dict.get)
skin_df['cell_type'] = skin_df['dx'].map(lesion_type_dict.get) 
skin_df['cell_type_idx'] = pd.Categorical(skin_df['cell_type']).codes

In [8]:
skin_df

,lesion_id,image_id,dx,dx_type,age,sex,localization,path,cell_type,cell_type_idx
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,/content/data/ISIC_0027419.jpg,Benign keratosis-like lesions,2
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,/content/data/ISIC_0025030.jpg,Benign keratosis-like lesions,2
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,/content/data/ISIC_0026769.jpg,Benign keratosis-like lesions,2
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,/content/data/ISIC_0025661.jpg,Benign keratosis-like lesions,2
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,/content/data/ISIC_0031633.jpg,Benign keratosis-like lesions,2
5,HAM_0001466,ISIC_0027850,bkl,histo,75.0,male,ear,/content/data/ISIC_0027850.jpg,Benign keratosis-like lesions,2
6,HAM_0002761,ISIC_0029176,bkl,histo,60.0,male,face,/content/data/ISIC_0029176.jpg,Benign keratosis-like lesions,2
7,HAM_0002761,ISIC_0029068,bkl,histo,60.0,male,face,/content/data/ISIC_0029068.jpg,Benign keratosis-like lesions,2
8,HAM_0005132,ISIC_0025837,bkl,histo,70.0,female,back,/content/data/ISIC_0025837.jpg,Benign keratosis-like lesions,2
9,HAM_0005132,ISIC_0025209,bkl,histo,70.0,female,back,/content/data/ISIC_0025209.jpg,Benign keratosis-like lesions,2


In [9]:
skin_df.isnull().sum()

lesion_id         0
image_id          0
dx                0
dx_type           0
age              57
sex               0
localization      0
path              0
cell_type         0
cell_type_idx     0
dtype: int64

In [0]:
skin_df['age'].fillna((skin_df['age'].mean()), inplace=True)

In [0]:
skin_df['image'] = skin_df['path'].map(lambda x: np.asarray(Image.open(x).resize((75,100))))

In [12]:
skin_df

,lesion_id,image_id,dx,dx_type,age,sex,localization,path,cell_type,cell_type_idx,image
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,/content/data/ISIC_0027419.jpg,Benign keratosis-like lesions,2,"[[[192, 155, 196], [190, 154, 190], [190, 147,..."
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,/content/data/ISIC_0025030.jpg,Benign keratosis-like lesions,2,"[[[25, 15, 26], [23, 12, 26], [29, 17, 37], [4..."
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,/content/data/ISIC_0026769.jpg,Benign keratosis-like lesions,2,"[[[188, 131, 140], [198, 135, 152], [199, 146,..."
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,/content/data/ISIC_0025661.jpg,Benign keratosis-like lesions,2,"[[[23, 10, 20], [29, 16, 26], [56, 29, 38], [7..."
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,/content/data/ISIC_0031633.jpg,Benign keratosis-like lesions,2,"[[[138, 93, 116], [153, 108, 131], [172, 126, ..."
5,HAM_0001466,ISIC_0027850,bkl,histo,75.0,male,ear,/content/data/ISIC_0027850.jpg,Benign keratosis-like lesions,2,"[[[4, 0, 1], [9, 0, 3], [11, 2, 5], [14, 3, 9]..."
6,HAM_0002761,ISIC_0029176,bkl,histo,60.0,male,face,/content/data/ISIC_0029176.jpg,Benign keratosis-like lesions,2,"[[[192, 146, 130], [194, 146, 142], [193, 146,..."
7,HAM_0002761,ISIC_0029068,bkl,histo,60.0,male,face,/content/data/ISIC_0029068.jpg,Benign keratosis-like lesions,2,"[[[151, 105, 89], [162, 120, 104], [162, 123, ..."
8,HAM_0005132,ISIC_0025837,bkl,histo,70.0,female,back,/content/data/ISIC_0025837.jpg,Benign keratosis-like lesions,2,"[[[103, 71, 72], [121, 79, 80], [135, 89, 91],..."
9,HAM_0005132,ISIC_0025209,bkl,histo,70.0,female,back,/content/data/ISIC_0025209.jpg,Benign keratosis-like lesions,2,"[[[226, 173, 189], [225, 173, 186], [230, 173,..."


In [14]:
# Checking the image size distribution
skin_df['image'].map(lambda x: x.shape).value_counts()

(100, 75, 3)    10015
Name: image, dtype: int64

In [0]:
features=skin_df['image']
target=skin_df['cell_type_idx']

In [0]:
x_train_o, x_test_o, y_train_o, y_test_o = train_test_split(features, target, test_size=0.20,random_state=1234)

In [0]:
x_train = np.asarray(x_train_o.tolist())
x_test = np.asarray(x_test_o.tolist())

In [0]:

# Perform one-hot encoding on the labels
y_train = to_categorical(y_train_o, num_classes = 7)
y_test = to_categorical(y_test_o, num_classes = 7)

In [0]:
x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size = 0.1, random_state = 2)

In [0]:
# Reshape image in 3 dimensions (height = 75px, width = 100px , canal = 3)
x_train = x_train.reshape(x_train.shape[0], *(75, 100, 3))
x_test = x_test.reshape(x_test.shape[0], *(75, 100, 3))
x_validate = x_validate.reshape(x_validate.shape[0], *(75, 100, 3))

In [27]:
# Set the CNN model 
# my CNN architechture is In -> [[Conv2D->relu]*2 -> MaxPool2D -> Dropout]*2 -> Flatten -> Dense -> Dropout -> Out
input_shape = (75, 100, 3)
num_classes = 7

model = Sequential()
model.add(Conv2D(8, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(16, (3, 3), activation='relu',padding = 'Same'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.40))

model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 73, 98, 8)         224       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 36, 49, 8)         0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 36, 49, 8)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 36, 49, 16)        1168      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 18, 24, 16)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 18, 24, 16)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 6912)             

In [0]:
# Define the optimizer
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [0]:
# Compile the model
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [0]:
# Fit the model
epochs = 5
batch_size = 10
history = model.fit(x_train,y_train, batch_size=None ,epochs = epochs, validation_data = (x_validate,y_validate), verbose = 1, steps_per_epoch=x_train.shape[0], validation_steps= 1,callbacks=None)

Train on 7210 samples, validate on 802 samples
Epoch 1/5
7210/7210 [==============================] - 8052s 1s/step - loss: 5.3027 - acc: 0.6709 - val_loss: 5.1449 - val_acc: 0.6808
Epoch 2/5
 358/7210 [>.............................] - ETA: 2:07:31 - loss: 5.2984 - acc: 0.6712